In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# Any results you write to the current directory are saved as output.

In [53]:

train_path = Path("D:/Kaggle/NLP/train.csv")
test_path = Path("D:/Kaggle/NLP/test.csv")
train=pd.read_csv(train_path)
test=pd.read_csv(test_path)



In [54]:
import re, string 
def processTweet(tweet):
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','',tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
    # Remove Punctuation and split 's, 't, 've with a space for filter
    tweet = re.sub(r'[' + string.punctuation + ']+', ' ', tweet)
    # Remove words with 2 or fewer letters
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ') 
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet

In [55]:
# clean dataframe's text column
train['text'] = train['text'].apply(processTweet)
test['text'] = test['text'].apply(processTweet)



In [67]:
train[train['target']==1]

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason this may allah forgiv...,1
1,4,NaN,NaN,forest fire near ronge sask canada,1
2,5,NaN,NaN,all residents asked shelter place are being no...,1
3,6,NaN,NaN,000 people receive evacuation orders california,1
4,7,NaN,NaN,just got sent this photo from ruby smoke from ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding bridge collapse into ...,1
7609,10870,NaN,NaN,the out control wild fires california even the...,1
7610,10871,NaN,NaN,utc 5km volcano hawaii,1
7611,10872,NaN,NaN,police investigating after bike collided with ...,1


In [68]:
X_train=train.text
y_train=train.target
X_test=test.text


In [77]:
from sklearn.pipeline import Pipeline                         #Pipe
#from sklearn.naive_bayes import MultinomialNB                # Training Naive Bayes (NB) classifier on training data.
from sklearn.feature_extraction.text import TfidfTransformer  #TF-IDF
from sklearn.feature_extraction.text import CountVectorizer   # Extracting features from text files
from lightgbm import LGBMClassifier

pipe = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2)))
                     , ('tfidf', TfidfTransformer())
                     , ('LGBMmodel', LGBMClassifier(n_estimators=3000,learning_rate=0.03))])



In [28]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'LGBMmodel__alpha': (1e-2, 1e-3),
             }

In [29]:
# do 10-fold cross validation for each of the 8 possible combinations of the above params
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, cv=10, param_grid=parameters, verbose=1)

In [31]:
grid.fit(X_train, y_train)
# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed: 26.5min finished



Best Model: 0.645081 using {'LGBMmodel__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}


Mean: 0.638250 Stdev:(0.050782) with: {'LGBMmodel__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Mean: 0.641797 Stdev:(0.060752) with: {'LGBMmodel__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Mean: 0.639695 Stdev:(0.060149) with: {'LGBMmodel__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Mean: 0.645081 Stdev:(0.060391) with: {'LGBMmodel__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
Mean: 0.638250 Stdev:(0.050782) with: {'LGBMmodel__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Mean: 0.641797 Stdev:(0.060752) with: {'LGBMmodel__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Mean: 0.639695 Stdev:(0.060149) with: {'LGBMmodel__alpha': 0.001, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Mean: 0.645081 Stdev:(0.060391) with: {'LGBMmodel__alpha'

In [78]:
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.03, max_depth=-1,
                                min_child_samples=20, min_child_weight=0.001,
                                min_spli

In [79]:
train_pred=pipe.predict(X_train)
from sklearn import metrics
metrics.confusion_matrix(y_train,train_pred)

array([[4270,   72],
       [ 247, 3024]], dtype=int64)

In [72]:
print(metrics.classification_report(y_train, train_pred, digits=3))

              precision    recall  f1-score   support

           0      0.902     0.966     0.933      4342
           1      0.951     0.861     0.904      3271

    accuracy                          0.921      7613
   macro avg      0.927     0.914     0.919      7613
weighted avg      0.923     0.921     0.921      7613



In [10]:
#X_test=X_test.as_matrix()


In [44]:

test_preds=pipe.predict(X_test)
output = pd.DataFrame({'id': test.id,
                       'target': test_preds})
output.to_csv('submission.csv', index=False)